In [1]:
from setting.setting_h2o2 import *

# System Config

In [2]:
sh = system_config_h2o2()

sh.dict['tiprack_1']['type'] = 'opentrons_96_tiprack_20ul'  
sh.dict['tiprack_2']['type'] = 'opentrons_96_tiprack_20ul' 
sh.dict['tiprack_3']['type'] = 'opentrons_96_tiprack_300ul'  
sh.dict['tiprack_4']['type'] = 'opentrons_96_tiprack_300ul' 
sh.dict['tiprack_1']['pos'] = 10 # for titration
sh.dict['tiprack_2']['pos'] = 11 # for titration and pre-estimation
sh.dict['tiprack_3']['pos'] = 1 # for samples(h2o2)
sh.dict['tiprack_4']['pos'] = 8 # for buffer(h2so4) and indicator 
sh.dict['reservoir_1']['pos'] = 9 # samples(h2o2) reservoir
sh.dict['reservoir_2']['pos'] = 6 # buffer(h2so4) and indicator reservoir
sh.dict['reservoir_3']['pos'] = 7 # titrant(KMno4) reservoir
sh.dict['reservoir_4']['pos'] = 4 # h2o reservoir
sh.dict['tiprack_1']['offset']['x'] = 0.10
sh.dict['tiprack_1']['offset']['y'] = 0.30
sh.dict['tiprack_1']['offset']['z'] = 0.20
sh.dict['tiprack_2']['offset']['x'] = 0.50
sh.dict['tiprack_2']['offset']['y'] = 0.20
sh.dict['tiprack_2']['offset']['z'] = 0.30
sh.dict['tiprack_3']['offset']['x'] = -0.70
sh.dict['tiprack_3']['offset']['y'] = 0.7
sh.dict['tiprack_3']['offset']['z'] = 0.10
sh.dict['tiprack_4']['offset']['x'] = -0.20
sh.dict['tiprack_4']['offset']['y'] = 0.90
sh.dict['tiprack_4']['offset']['z'] = 0.00
sh.dict['plate_1']['offset']['x'] = -0.10
sh.dict['plate_1']['offset']['y'] = 0.20
sh.dict['plate_1']['offset']['z'] = 0.80
sh.dict['pipette_1']['type'] = 'p300_multi_gen2'
sh.dict['pipette_1']['mount']='left'
sh.dict['pipette_2']['type'] = 'p20_single_gen2'
sh.dict['pipette_2']['mount']='right'
sh.save_json()

## Load from File to SMB server

In [ ]:
path_json_dump= os.path.abspath(r'../DriveSMB')
sh2 = sh.load_from_json(path_json_dump,'ot2_system_config_h2o2.json')
sh2.save_json(path_json_dump, 'test_h2o2.json') # use this code to test the connection between working PC and ot2
sh2

# Transfer Config

In [ ]:
tc = transfer_config_h2o2()
# Note: volume number must be between the minimum and the maximum range of the used piptte
# Transfer 1 : Buffer or Indicator addition(H2SO4), Multi-Channel Pipette p300, tiprack_4
tc.dict['transfer_1']['tiprack'] = ['A1:A12']
tc.dict['transfer_1']['vol'] = {'A1':100} # volume aspirate uL
tc.dict['transfer_1']['locs'] = [['A1','A7:A8']] # buffer + indicator in all cols
tc.dict['transfer_1']['blowout'] = True 
tc.dict['transfer_1']['mix'] = None


# Transfer 2 : Sample Addition(H2O2), Multi-Channel Pipette p300, tiprack_3
tc.dict['transfer_2']['tiprack'] = ['A1:A12']
tc.dict['transfer_2']['vol'] = 100 # volume aspirate uL
tc.dict['transfer_2']['locs'] = ['A3:A4','A7:A8'] 
tc.dict['transfer_2']['blowout'] = True 
tc.dict['transfer_2']['mix'] = None


# Transfer 3 : Pre-estimation(titrant), Single-Channel Pipette p20, tiprack_2: E11-H11, only number can change
# Note: the input volume is 20 uL, but within the titration_general.py, this aspiration was operated 2 times, 
# 40 uL in total for each concentration of titrant
# select the condition parameter, including a_star and hue (hue_threshold and direction)
tc.dict['transfer_3']['tiprack'] = ['E11:H11']
tc.dict['transfer_3']['vol'] = 20 # volume aspirate uL
tc.dict['transfer_3']['locs'] = [['A1','E7:E8'],
                                 ['A2','F7:F8'],
                                 ['A3','G7:G8'],
                                 ['A4','H7:H8']]
tc.dict['transfer_3']['mix'] = {'rep' : 0, 'vol' : 20}
tc.dict['transfer_3']['blowout'] = True
#tc.dict['transfer_3']['param'] = 'a*' 
tc.dict['transfer_3']['params'] = 'hue'
tc.dict['transfer_3']['boundary'] = 50  # Threshold used to identify the titration endpoint (a*: 2; pH: 50; wha: 230)
tc.dict['transfer_3']['larger_than_boundary'] = False # Boolean(type) indicating whether the first data point is higher or lower than the boundary

# Transfer 4 : Titration(titrant), Single-Channel Pipette p20, tiprack_1
tc.dict['transfer_4']['tiprack'] = ['A1:H12','A1:H12']
tc.dict['transfer_4']['vol'] = 4 # volume aspirate uL
tc.dict['transfer_4']['locs'] = [['A3','A4'],['A7:A8']]
tc.dict['transfer_4']['titration'] = [15] * len(tc.dict['transfer_4']['locs'][0]) 
tc.dict['transfer_4']['mix']['rep'] = 0
tc.dict['transfer_4']['mix']['vol'] = 20
tc.dict['transfer_4']['blowout'] = True 
tc.dict['transfer_4']['speed'] = {'aspirate' : 1, 'dispense' : 1}
tc.save_json()
tc